In [1]:
import argparse, os
import cv2
import numpy as np
import imageio
import time

In [2]:
def readPicture(picturepath):
    #  open ImageObject
    img = cv2.imread(picturepath, cv2.IMREAD_UNCHANGED)#cv2.IMREAD_UNCHANGED is important that the output is (x,y,ChannelRGB)
    return(img)

In [3]:
def savePic(picture,fileName,extention,outPath):
    outPath = outPath+fileName+'.'+extention # combines the path with the name and extention of the file
    print('Saving to ===> '+outPath)
    try:
        #imageio.imwrite(outPath,picture,format=extention)# old way
        cv2.imwrite(outPath,picture)#saves Pictures 
    except:
        print('Failed while saving picture: '+fileName+' to '+ outPath+' sorry :(') #writes an error
        print('--------------------')

In [4]:
def YUVtoRGB(img):
    pictureRGB = cv2.cvtColor(img, cv2.COLOR_YUV2BGR, cv2.IMREAD_UNCHANGED) #uses the CV2 method to convert the color space from YU-V to RGB
    #pictureYUV = cv2.cvtColor(img, cv2.COLOR_BGR2YUV, cv2.IMREAD_UNCHANGED)
    return pictureRGB

In [5]:
#toDo Add parser with args
###### Imput section
print('Pictures in the folder need to have the format: [number]-[y]or[u]or[v].png e.g. : 28-y_pred.png,28-u_pred.png,28-v_pred.png')
print('FILENAME MUST HAVE the format: 23-y_pred !')
path = input('Path to pictures who should be converted defaut: ./yuvPic/: ') or './yuvPic/'
inputextention = input('What fileextention do the to read pictures have? [default: png]') or 'png'
outputextention = input('Please type outputextention[default: hdr]: ') or 'hdr'
outputpath = input('Where to write the stiched pictures to? [default: ./hdrOut/]: ') or './hdrOut/'
namePic = input('What should be the name of the stiched pictures? [default: predictedPic]') or 'predictedPic'
wantYUV = input('Do you want to keep YUV color space type y? [deflaut: no (RGB_color space)]') or 'no'
aOp = sum(1 for f in os.listdir(path) if f.endswith('.'+inputextention)) #summ all ending with extention

Pictures in the folder need to have the format: [number]-[y]or[u]or[v].png e.g. : 28-y_pred.png,28-u_pred.png,28-v_pred.png
FILENAME MUST HAVE the format: 23-y_pred !
Path to pictures who should be converted defaut: ./yuvPic/: /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/JSI-GAN_x2_exp1/
What fileextention do the to read pictures have? [default: png]
Please type outputextention[default: hdr]: 
Where to write the stiched pictures to? [default: ./hdrOut/]: /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/
What should be the name of the stiched pictures? [default: predictedPic]
Do you want to keep YUV color space type y? [deflaut: no (RGB_color space)]


In [12]:
#Working Version 10022021
start_time = time.time() #start Timer
#TO DO Parser
#desc ='yuv to RGB'
#parser = argparse.ArgumentParser(description=desc)
#parser.add_argument('--yuv', type=str, default='./', help='path to Folder of yuv images')
#print(parser.parse_args())
#what, b = parser.parse_known_args()

#if what.yuv == './' :
 #   print('yes')
start_time = time.time()
i = 0
#print(aOp)

while (i <= aOp-1 ): # read y
    try:
        if ((str(os.listdir(path)[i]).split('-')[1]).split('_')[0]) == 'y': # only searching for y picitures
            print('Progress: '+str(int((i/aOp)*100))+' % there are '+str(aOp-i)+' pictures to go')
            name = os.listdir(path)[i] #finding the Name
            #print(name +' should be the Y')
            picpath = path + name #combining Name and path
            
            #####Y
            picy = readPicture(picpath) #reads a pic y to find the x,y axes should be the same for all pictures
            yuvPic = np.zeros((int(picy.shape[0]),int(picy.shape[1]),3)) # generates the x and y achses and channels of picture
            yuvPic[:,:,0] = picy # packs the Y in pos 1
            num = (str(os.listdir(path)[i]).split('-')[0])#.split('_')[0]
            newPic = path + num + '-u_'+name.split('_')[1]
            
            #####U
            picu = readPicture(newPic)#reads a picture with Ending U
            yuvPic[:,:,1] = picu # packs the u in pos 0
            newPic = path + num + '-v_'+name.split('_')[1] #Generates the Name for v
            
            #####V
            picv = readPicture(newPic)#reads the new picture with ending V
            yuvPic[:,:,2] = picv # packs the u in pos 2
            #Right for the Testdata ist: 0y2u1v <<<<<<<<<<<<<

            if(wantYUV != 'y'):
                number = int(os.listdir(path)[i].split('-')[0])-1
                yuvPic_hdr = YUVtoRGB((yuvPic/((2**10)-1)).astype(np.float32)) #changes the uint16 into float32 and from YUV to RGB
                savePic(((yuvPic_hdr).astype(np.float32)),(str(number)+namePic),outputextention,outputpath) #saves final RGB pic
                
                helpPath = outputpath+'rgb/'
                if not os.path.exists(helpPath):
                    os.mkdir(helpPath)
                rgbPNG= np.clip((yuvPic_hdr*((2**16)-1)), 0, ((2**16)-1)).astype(np.uint16)
                number = int(os.listdir(path)[i].split('-')[0])-1
                savePic(rgbPNG,(str(number).zfill(6)),'png',helpPath)
                
            if(wantYUV == 'y'):
                yuvPic = YUVtoRGB(yuvPic)
                savePic(yuvPic,(str(i)+namePic),outputextention,outputpath)#saves final YUV pic
    except:
        print('')
    i = i + 1

print("--- %s seconds ---" % (time.time() - start_time))
print(str((time.time() - start_time)/60))
print('------------------------- Done --------------------')

Progress: 0 % there are 1314 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/3predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000193.png
Progress: 0 % there are 1311 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/6predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000234.png
Progress: 0 % there are 1305 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/12predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000201.png
Progress: 0 % there are 1304 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/13predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000061.png
Progress: 1 % ther

Progress: 8 % there are 1207 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/110predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000089.png
Progress: 8 % there are 1202 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/115predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000385.png
Progress: 8 % there are 1200 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/117predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000291.png
Progress: 9 % there are 1198 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/119predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000025.png
Progress: 9 

Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/213predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000273.png
Progress: 16 % there are 1099 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/218predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000356.png
Progress: 16 % there are 1098 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/219predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000013.png
Progress: 16 % there are 1096 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/221predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000434.png
Progress: 16 % there are 1095 pictures to go
Saving t

Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000171.png
Progress: 23 % there are 1007 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/310predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000278.png
Progress: 23 % there are 1001 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/316predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000018.png
Progress: 24 % there are 1000 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/317predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000126.png
Progress: 24 % there are 999 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/318predictedPic.hdr
Saving to

Progress: 30 % there are 909 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/408predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000422.png
Progress: 31 % there are 908 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/409predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000001.png
Progress: 31 % there are 905 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/412predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000340.png
Progress: 31 % there are 902 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/415predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000012.png
Progress: 31

...

Progress: 68 % there are 410 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/907predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000188.png
Progress: 68 % there are 409 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/908predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000197.png
Progress: 69 % there are 407 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/910predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000090.png
Progress: 69 % there are 404 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/913predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000386.png
Progress: 69

Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000353.png
Progress: 76 % there are 314 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/1003predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000128.png
Progress: 76 % there are 312 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/1005predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000016.png
Progress: 76 % there are 311 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/1006predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000435.png
Progress: 76 % there are 308 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/1009predictedPic.hdr
Saving t

Progress: 84 % there are 204 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/1113predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000063.png
Progress: 84 % there are 200 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/1117predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000326.png
Progress: 85 % there are 197 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/1120predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000309.png
Progress: 85 % there are 194 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/1123predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000052.png
Progress

Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/1207predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000262.png
Progress: 92 % there are 103 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/1214predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000253.png
Progress: 92 % there are 102 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/1215predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000414.png
Progress: 92 % there are 100 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/1217predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000033.png
Progress: 92 % there are 99 pictures to go
Saving to

Progress: 98 % there are 15 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/1302predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000129.png
Progress: 99 % there are 5 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/1312predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000317.png
Progress: 99 % there are 3 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/1314predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000164.png
Progress: 99 % there are 2 pictures to go
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/1315predictedPic.hdr
Saving to ===> /Users/littledragon/Documents/BA 13022020/resultate/JSI-GAN/reinhard/rgb/000151.png
--- 477.7205698

In [7]:
#yuvPic

In [8]:
#rgbPic = YUVtoRGB(())

In [9]:
#print(str((i/aOp)*100)+'%')

100.0%


In [10]:
#os.listdir(path)[i].split('-')[0]

'png'